# **Project 1: Analysis of LinkedIn Job Postings for 2024**

This project aims to explore and analyze data from LinkedIn job postings, focusing on the year 2024. The dataset, sourced from **[Kaggle](https://www.kaggle.com/datasets/arshkon/linkedin-job-postings)**, consists of 11 CSV files containing detailed information on job postings, including company names, industries, specialties, skill requirements, salaries, job types, and more. 

The primary objective is to clean and transform the datasets, then store them in a well-structured MySQL relational database. This ensures data integrity, efficient querying, and seamless integration with analytical tools. 

Once the data is stored, it will be queried and analyzed to uncover key insights, which will be visualized and shared using Microsoft Power BI. The goal of this project is to create an interactive dashboard, offering valuable insights into the job market trends, skill demands, and industry distribution for LinkedIn job postings in 2024.

In [28]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [29]:
# Specify the path to the folder containing your data files
data_folder = os.path.expanduser('data')

# Load all the data files in the specified folder
dataframes = {} # Dictionary to hold all Dataframes

for filename in os.listdir(data_folder): # Check for Excel files
    if filename.endswith('.csv'):
        file_path = os.path.join(data_folder, filename)
        df = pd.read_csv(file_path)
        dataframes[filename] = df # Store the filename as a key and Dataframe as value


In [30]:
# Display the loaded DataFrames
for name, df in dataframes.items():
    print(f"Data loaded from {name}:")

Data loaded from benefits.csv:
Data loaded from companies.csv:
Data loaded from company_industries.csv:
Data loaded from company_specialities.csv:
Data loaded from employee_counts.csv:
Data loaded from industries.csv:
Data loaded from job_industries.csv:
Data loaded from job_skills.csv:
Data loaded from postings.csv:
Data loaded from salaries.csv:
Data loaded from skills.csv:


# The Data
## benefits.csv
The `benefits.csv` file is a dimension table that contains information about benefits like `medical insurance` for job postings. It contains `job_id` which is a primary key, `inferred` and `type`. Also, it does not contain any missing values.

In [31]:
# Print the first few row of benefits.csv
dataframes['benefits.csv'].head()

,job_id,inferred,type
0,3887473071,0,Medical insurance
1,3887473071,0,Vision insurance
2,3887473071,0,Dental insurance
3,3887473071,0,401(k)
4,3887473071,0,Student loan assistance


In [32]:
# Check for missing values
dataframes['benefits.csv'].isnull().sum()

job_id      0
inferred    0
type        0
dtype: int64

## companies.csv
The `companies.csv` file is a dimension table that conatins information about companies such as `address` and `description`. Some records in the data are missing attributes such as `description` and `company_size`. These missing values are going to be ignored for now.

In [33]:
# Print the first few row of companies.csv
dataframes['companies.csv'].head()

,company_id,name,description,company_size,state,country,city,zip_code,address,url
0,1009,IBM,"At IBM, we do more than work. We create. We cr...",7.0,NY,US,"Armonk, New York",10504,International Business Machines Corp.,https://www.linkedin.com/company/ibm
1,1016,GE HealthCare,Every day millions of people feel the impact o...,7.0,0,US,Chicago,0,-,https://www.linkedin.com/company/gehealthcare
2,1025,Hewlett Packard Enterprise,Official LinkedIn of Hewlett Packard Enterpris...,7.0,Texas,US,Houston,77389,1701 E Mossy Oaks Rd Spring,https://www.linkedin.com/company/hewlett-packa...
3,1028,Oracle,We’re a cloud technology company that provides...,7.0,Texas,US,Austin,78741,2300 Oracle Way,https://www.linkedin.com/company/oracle
4,1033,Accenture,Accenture is a leading global professional ser...,7.0,0,IE,Dublin 2,0,Grand Canal Harbour,https://www.linkedin.com/company/accenture


In [34]:
# Check for missing values
dataframes['companies.csv'].isnull().sum()

company_id         0
name               1
description      297
company_size    2774
state             22
country            0
city               1
zip_code          28
address           22
url                0
dtype: int64

## company_industries.csv
This file is a dimension table that contains information about the industry that each company belongs to. `company_id` is a primary key.

In [35]:
# Print the first few row of company_industries.csv
dataframes['company_industries.csv'].head()

,company_id,industry
0,391906,Book and Periodical Publishing
1,22292832,Construction
2,20300,Banking
3,3570660,Book and Periodical Publishing
4,878353,Staffing and Recruiting


In [36]:
# Check for missing values
dataframes['company_industries.csv'].isnull().sum()

company_id    0
industry      0
dtype: int64

## company_industries.csv
This file is a dimension table that contains information about the speciality of each company. `company_id` is a primary key. It does not contain any missing values.

In [37]:
# Print the first few row of company_specialisations.csv
dataframes['company_specialities.csv'].head()

,company_id,speciality
0,22292832,window replacement
1,22292832,patio door replacement
2,20300,Commercial Banking
3,20300,Retail Banking
4,20300,Mortgage


In [38]:
# Check for missing values
dataframes['company_specialities.csv'].isnull().sum()

company_id    0
speciality    0
dtype: int64

# postings.csv
The psotings.csv is the main fact table. It contains `123849` records(rows) and `31` attributes(columns). It contains job posting scraped from the linkedin website and this is the main table that would be used for the data analysis. Each job posting has a lot of attributes some of which are useful and some are not very useful. The primary key in the table is `job_id`.

In [39]:
# Print the number of rows and columns
dataframes['postings.csv'].shape

(123849, 31)

In [40]:
# Print a lsit of columns in 'postings.csv'
dataframes['postings.csv'].columns

Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
       'pay_period', 'location', 'company_id', 'views', 'med_salary',
       'min_salary', 'formatted_work_type', 'applies', 'original_listed_time',
       'remote_allowed', 'job_posting_url', 'application_url',
       'application_type', 'expiry', 'closed_time',
       'formatted_experience_level', 'skills_desc', 'listed_time',
       'posting_domain', 'sponsored', 'work_type', 'currency',
       'compensation_type', 'normalized_salary', 'zip_code', 'fips'],
      dtype='object')

### *Removing missing values*
To deal with missing values, columns that contain a 70% of null values will be removed to ensure that the data is at the highest quality.

In [41]:
# Print the proportion of missing values in each column
dataframes['postings.csv'].isnull().sum() / dataframes['postings.csv'].shape[0]

job_id                        0.000000
company_name                  0.013880
title                         0.000000
description                   0.000057
max_salary                    0.759441
pay_period                    0.708734
location                      0.000000
company_id                    0.013864
views                         0.013638
med_salary                    0.949293
min_salary                    0.759441
formatted_work_type           0.000000
applies                       0.811706
original_listed_time          0.000000
remote_allowed                0.876898
job_posting_url               0.000000
application_url               0.296046
application_type              0.000000
expiry                        0.000000
closed_time                   0.991336
formatted_experience_level    0.237459
skills_desc                   0.980307
listed_time                   0.000000
posting_domain                0.322716
sponsored                     0.000000
work_type                

In [42]:
# Removing columns with a lot missing values
columns_to_remove = ['max_salary', 'pay_period', 'med_salary', 'min_salary', 'applies', 'remote_allowed',
                     'application_url', 'closed_time', 'formatted_experience_level', 'skills_desc', 'currency', 
                     'compensation_type', 'normalized_salary', 'original_listed_time']

dataframes['postings.csv'] = dataframes['postings.csv'].drop(columns=columns_to_remove)

In [43]:
# Display the postings.csv table with only necessary columns
dataframes['postings.csv'].drop(columns=['title', 'description', 'job_posting_url', 'posting_domain']).head()

,job_id,company_name,location,company_id,views,formatted_work_type,application_type,expiry,listed_time,sponsored,work_type,zip_code,fips
0,921716,Corcoran Sawyer Smith,"Princeton, NJ",2774458.0,20.0,Full-time,ComplexOnsiteApply,1.715990e+12,1.713398e+12,0,FULL_TIME,8540.0,34021.0
1,1829192,NaN,"Fort Collins, CO",NaN,1.0,Full-time,ComplexOnsiteApply,1.715450e+12,1.712858e+12,0,FULL_TIME,80521.0,8069.0
2,10998357,The National Exemplar,"Cincinnati, OH",64896719.0,8.0,Full-time,ComplexOnsiteApply,1.715870e+12,1.713278e+12,0,FULL_TIME,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP","New Hyde Park, NY",766262.0,16.0,Full-time,ComplexOnsiteApply,1.715488e+12,1.712896e+12,0,FULL_TIME,11040.0,36059.0
4,35982263,NaN,"Burlington, IA",NaN,3.0,Full-time,ComplexOnsiteApply,1.716044e+12,1.713452e+12,0,FULL_TIME,52601.0,19057.0


### *Transforming columns*
The columns `expiry` and `listing_time` are columns that could potentially be useful. They are supposed to be Date/Time data but here they are stored as integers. These values would need to be converted into Date/Time.

In [44]:
# Convert 'expiry' and 'listing_time' columns from integers to Date/Time
dataframes['postings.csv']['expiry'] = pd.to_datetime(dataframes['postings.csv']['expiry'], unit='ms')
dataframes['postings.csv']['listed_time'] = pd.to_datetime(dataframes['postings.csv']['listed_time'], unit='ms')

## Storing in a database
The reason for storing in a database is to ensure data efficiecy and integrity. Each pandas dataframe in `dataframes` will be stored as a table in the `jobListings` database using the `to_sql` method. 

In [45]:
import pymysql
from sqlalchemy import create_engine

In [3]:
# set database parameters
from getpass import getpass
username = 'root'
password = getpass() # requires user input to get password
host = 'localhost'
port = '3306'
database_name = 'jobListings' #database name, this was created manually using MySQL command line 

 ········


In [47]:
# Create a database connection engine
connection_string = f'mysql+mysqlconnector://{username}:{password}@{host}/{database_name}'

engine = create_engine(connection_string)

In [48]:
# Save each Dataframe to the database
for table_name, df in dataframes.items():
    df.to_sql(name=table_name, con=engine, if_exists='replace', index=False, chunksize=1000)
    print(f"{table_name} saved to the MySQL database successfully!")


benefits.csv saved to the MySQL database successfully!
companies.csv saved to the MySQL database successfully!
company_industries.csv saved to the MySQL database successfully!
company_specialities.csv saved to the MySQL database successfully!
employee_counts.csv saved to the MySQL database successfully!
industries.csv saved to the MySQL database successfully!
job_industries.csv saved to the MySQL database successfully!
job_skills.csv saved to the MySQL database successfully!
postings.csv saved to the MySQL database successfully!
salaries.csv saved to the MySQL database successfully!
skills.csv saved to the MySQL database successfully!


### *Data Quering*
Now that the data is stored in a MySQL databases, it is possible to write SQL queries from Jupyter Notebook. It is a good idea to check for data entry error especially in for numerical data. The query below gets the `med_salary`, `min_salary` and `max_salary` of job postings in descending order. The result shows that there is an error in one of the entry. A job postings with a salary of `120 Million` is very unlikely on LinkedIn. This entry will be removed from the database.

In [49]:
query = """
SELECT `salaries.csv`.max_salary, `salaries.csv`.med_salary, `salaries.csv`.min_salary, job_id
FROM `salaries.csv`
ORDER BY `salaries.csv`.max_salary DESC
LIMIT 10
"""


In [50]:
df_query_result = pd.read_sql(query, con=engine)

# Display the result
df_query_result.head()


,max_salary,med_salary,min_salary,job_id
0,120000000.0,None,85000000.0,3904911743
1,1500000.0,None,350000.0,3884921825
2,1500000.0,None,350000.0,3884923184
3,1500000.0,None,350000.0,3884920734
4,1500000.0,None,350000.0,3884925602


In [51]:
query = """
DELETE FROM `postings.csv`
WHERE job_id = 3904911743;
"""


In [52]:
# df_query_result = pd.read_sql(query, con=engine)

# Unfortunately, can't delete record from a database without permission so this will be done manually in MySQL command line